# Dreadnode Image Logging

This notebook demonstrates how to log images using Dreadnode's `Image` data type. The examples cover various image formats and sources including file paths, PIL images, numpy arrays, and base64 encoded strings.

## Features

- Log images from file paths (JPG, PNG, etc.)
- Convert and log PIL Image objects
- Transform numpy arrays into images
- Handle raw bytes and base64 encoded images
- Convert between image modes (RGB, RGBA, grayscale)

⚠️ Note: Ensure you have installed the multimodal extras to use the Video data type:
`pip install dreadnode[multimodal]`

In [ ]:
import dreadnode as dn

dn.configure(
    server="Your Dreadnode API", # Replace with your server address
    token="Your Dreadnode API Key", # Replace with your token
    project="image-examples"
)

## 1. File Path Examples

Let's first look at logging images directly from file paths. We'll create a temporary image file to use in our examples.

In [ ]:
from dreadnode import Image
from PIL import Image as PILImage
import numpy as np
import tempfile
import os

temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
temp_file.close()
image_file_path = temp_file.name

# Generate a simple gradient test image
width, height = 300, 200
img_array = np.zeros((height, width, 3), dtype=np.uint8)
for x in range(width):
    for y in range(height):
        r = int(255 * x / width)
        g = int(255 * y / height)
        b = int(255 * (x + y) / (width + height))
        img_array[y, x] = [r, g, b]
        
# Save the generated image
test_img = PILImage.fromarray(img_array)
test_img.save(image_file_path)

print(f"Created test image at: {image_file_path}")

with dn.run("file_path_example") as r:
    # Basic file path logging
    dn.log_input("image_file", Image(image_file_path, caption="RGB Gradient Example"))
    
    # Load and convert to grayscale
    dn.log_input("grayscale_image", Image(image_file_path, mode="L", caption="Grayscale Conversion"))
    
    # Explicit format override
    dn.log_input("format_override", Image(image_file_path, format="jpg", caption="Format override to JPG"))

## 2. PIL Image Objects

Dreadnode can directly log PIL Image objects, which allows you to perform image processing before logging.

In [ ]:
with dn.run("pil_image_example") as r:
    
    pil_img = PILImage.open(image_file_path)
    
    # Log the original PIL image
    dn.log_input("original_pil", Image(pil_img, caption="Original PIL Image"))
    
    # Convert to grayscale
    grayscale = pil_img.convert("L")
    dn.log_input("pil_grayscale", Image(grayscale, caption="Grayscale PIL Image"))
    
    # Apply rotation
    rotated = pil_img.rotate(45, expand=True)
    dn.log_input("pil_rotated", Image(rotated, caption="Rotated 45 degrees"))
    
    # Resize
    resized = pil_img.resize((150, 100))
    dn.log_input("pil_resized", Image(resized, caption="Resized to 150×100"))
    
    # Convert to RGBA (with transparency)
    rgba = pil_img.convert("RGBA")
    # Add transparency to the top half
    data = np.array(rgba)
    data[:data.shape[0]//2, :, 3] = 128  # 50% transparency to top half
    rgba_modified = PILImage.fromarray(data)
    dn.log_input("pil_rgba", Image(rgba_modified, format="png", caption="RGBA with transparency"))

## 3. Numpy Arrays

Numpy arrays are commonly used for image representation in machine learning. Dreadnode can log numpy arrays as images in various formats.

In [ ]:
with dn.run("image_numpy_test") as r:
    
    # 3.1 RGB Array (channels last - HWC format)
    rgb_array = np.random.randint(0, 255, (200, 200, 3), dtype=np.uint8)
    dn.log_input("numpy_rgb", Image(rgb_array, caption="Random RGB Array (200×200×3)"))
    
    # 3.2 Grayscale Array (2D)
    gray_array = np.random.randint(0, 255, (200, 200), dtype=np.uint8)
    dn.log_input("numpy_gray", Image(gray_array, caption="Random Grayscale Array (200×200)"))
    
    # 3.3 RGBA Array with transparency
    rgba_array = np.zeros((200, 200, 4), dtype=np.uint8)
    # Create a red square with 50% transparency
    rgba_array[50:150, 50:150, 0] = 255  # Red channel
    rgba_array[50:150, 50:150, 3] = 128  # Alpha channel (50%)
    dn.log_input("numpy_rgba", Image(rgba_array, format="png", caption="RGBA with transparent red square"))
    
    # 3.4 Float array [0-1] range
    float_array = np.zeros((200, 200, 3), dtype=np.float32)
    # Make a color gradient
    for i in range(200):
        float_array[:, i, 0] = i / 200.0  # Red increases from left to right
        float_array[i, :, 1] = i / 200.0  # Green increases from top to bottom
    dn.log_input("numpy_float", Image(float_array, caption="Float gradient (0-1 range)"))
    
    # 3.5 Channels first format (PyTorch style - CHW)
    chw_array = np.random.randint(0, 255, (3, 200, 200), dtype=np.uint8)
    dn.log_input("numpy_chw", Image(chw_array, caption="Channels-first array (3×200×200)"))

## 4. Raw Bytes and Binary Data

Images can also be logged from raw bytes, which is useful when working with image data from APIs or databases.

In [ ]:
import io

with dn.run("bytes_test") as r:
    # Create a simple test image to use for our examples
    pil_img = PILImage.new("RGB", (100, 100), color="red")
    
    # 4.1 Basic PNG bytes
    buffer = io.BytesIO()
    pil_img.save(buffer, format="PNG")
    png_bytes = buffer.getvalue()
    dn.log_input("bytes_png", Image(png_bytes, format="png", caption="PNG bytes (red square)"))
    
    # 4.2 JPEG bytes
    buffer = io.BytesIO()
    pil_img.save(buffer, format="JPEG", quality=90)
    jpeg_bytes = buffer.getvalue()
    dn.log_input("bytes_jpeg", Image(jpeg_bytes, format="jpeg", caption="JPEG bytes (red square)"))
    
    # 4.3 Bytes with mode conversion
    dn.log_input("bytes_grayscale", Image(png_bytes, format="png", mode="L", caption="PNG bytes converted to grayscale"))

## 5. Base64 Encoded Images

Base64 encoded images are common in web applications and APIs. Dreadnode supports Data URLs and raw base64 strings.

In [ ]:
import base64

with dn.run("base64_test") as r:
    pil_img = PILImage.new("RGB", (100, 100), color="blue")
    
    # Save as PNG
    buffer = io.BytesIO()
    pil_img.save(buffer, format="PNG")
    png_bytes = buffer.getvalue()
    
    # Create base64 string with data URL
    png_base64 = base64.b64encode(png_bytes).decode('utf-8')
    png_data_url = f"data:image/png;base64,{png_base64}"
    
    # 5.1 Log with data URL format
    dn.log_input("base64_dataurl", Image(png_data_url, caption="Data URL format (blue square)"))
    
    # 5.2 Log with grayscale conversion
    dn.log_input("base64_grayscale", Image(png_data_url, mode="L", caption="Data URL converted to grayscale"))

## Summary

In this notebook, we've demonstrated how to log images in Dreadnode from various sources:

1. File paths (e.g., JPG, PNG files)
2. PIL Image objects
3. Numpy arrays in different formats
4. Raw bytes and binary data
5. Base64 encoded strings

We've also shown how to apply transformations like format conversion, grayscale conversion, and resizing before logging.
